# Instagram Crawler

In [ ]:
from bs4 import BeautifulSoup as bs 
import pandas as pd
import requests
import re
import os
import random

In [ ]:
URL = 'https://dogtime.com/dog-breeds/profiles'

HEADERS = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
}

page = requests.get(URL)
soup = bs(page.content, 'html.parser')

## Get All The Dog Breeds

In [ ]:
#group by letter - gbl
dogs_breeds = []
for gbl in soup.find_all('div',{'class': 'article-crumbs'}):
    for breed in gbl.find_all('a', {'class': 'list-item-title'}):
        dogs_breeds.append(breed.get_text().replace("-", " ").upper())

## Insert To DataFrame

In [ ]:
breeds_path = 'C:/Users/Asus/Data Science Project/Stansford Dataset/images/Images'
breeds_stansford = []
for filename in os.listdir(breeds_path):
        word = filename[filename.find('-'):][1:].replace("_", " ")
        word = word.replace("-", " ")
        word= word[:1].upper() + word[1:]
        breeds_stansford.append(word)

In [ ]:
data = {'breed':breeds_stansford}
df = pd.DataFrame(data)
df

## Sepcific Breeds

In [ ]:
final_breeds = []

path = os.getcwd()
my_file = open(os.path.join(path ,"Breeds_File.txt"), "r")
content = my_file.read()
content_list = content.split("\n")
my_file.close()

for breed in content_list:
        word = breed.replace("_", " ").replace("-", " ").replace(" ", "")
        word= word[:1].upper() + word[1:]
        final_breeds.append(word)

# Crawling Instagram With Selenium

## Log in Instagram account

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import wget
import time

In [ ]:
#set env var
os.environ['insta_username'] = "****"
os.environ['insta_password'] = "****"

In [ ]:
#driver = webdriver.Chrome('C:/Users/Asus/Documents/WebDriver/chromedriver.exe')
s = Service('C:/Users/Asus/Documents/WebDriver/chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get('https://www.instagram.com/')

In [ ]:
#wait for a maximum of 10 seconds for an element
username = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
password.clear()

username.send_keys(os.environ['insta_username'])
password.send_keys(os.environ['insta_password'])
time.sleep(7)
log_in = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

time.sleep(5)
not_now = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Not Now')]"))).click()
#not now 2 for notification settings
time.sleep(5)
not_now2 = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Not Now')]"))).click()

## Breeds by Hashtags

### Base Funcitons

In [ ]:
def create_folder(query):
    if not os.path.exists(os.getcwd() + '/' + query):
        path = os.getcwd()
        path = os.path.join(path, query)
        os.mkdir(path)
        print(query + " Folder Created")
    else:
        print(query + " Folder Already Exist")
        

def save_images(images, query):
    counter = 0
    path = os.getcwd()
    path = os.path.join(path, query)
    for img in images:
        save_as = os.path.join(path ,query + str(counter) +'.jpg')
        wget.download(img, save_as)
        counter += 1
    
    print("%s Images Saved! | Total of: %s Images" %(query,counter))

### Crawling

In [ ]:
time.sleep(5)

wait = WebDriverWait(driver, 10)

for breed in final_breeds:
    
    img_urls = []
    query = breed.replace(" ", "")
    page = driver.get("https://www.instagram.com/explore/tags/" + query)

    #scroll
    element = wait.until(EC.element_to_be_clickable((By.TAG_NAME, 'img')))
    scrolldown = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var scrolldown=document.body.scrollHeight;return scrolldown;")
    #window.scrollTo(0, document.body.scrollHeight);
    count_scroll=0 
    match=False
    count_opens=0
    
    while(match==False):
        element = wait.until(EC.element_to_be_clickable((By.TAG_NAME, 'img')))
        time.sleep(random.randint(2,4))
        images = driver.find_elements(By.TAG_NAME,'img')
        time.sleep(random.randint(2,4))
        images = [image.get_attribute('src') for image in images]
        img_urls.extend(images)
        #Remove Duplicates Of Img URLs
        img_urls = list( dict.fromkeys(img_urls))
        images.clear()
        print("Images: %s | Counter: %s" % (len(img_urls), count_scroll))
        time.sleep(random.randint(3,4))
        
        last_count = scrolldown
        scrolldown = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var scrolldown=document.body.scrollHeight;return scrolldown;")
        
        if(last_count==scrolldown):
            time.sleep(random.randint(2,4))     
            imgclick = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#react-root > section > main > article > div:nth-child(3) > div > div:nth-child(11) > div:nth-child(1)"))).click()            
            time.sleep(random.randint(7,10))                                
            not_now = driver.find_element_by_xpath('//div[@class="QBdPU "]/*[name()="svg"][@aria-label="Close"]').click()
            
            
        if(random.randint(0, 6)==6):
            time.sleep(random.randint(2,4))     
            imgclick = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#react-root > section > main > article > div:nth-child(3) > div > div:nth-child(11) > div:nth-child(1)"))).click()            
            time.sleep(random.randint(7,10))                                
            not_now = driver.find_element_by_xpath('//div[@class="QBdPU "]/*[name()="svg"][@aria-label="Close"]').click()
       
       
        
        if ( (count_scroll==835) or (len(img_urls)>10000)):
            match=True
        count_scroll+=1
    
    #create path if dosent exsist
    create_folder(query)
    
    #save the images
    save_images(img_urls, query)
    
    #sleep dealy between get request
    time.sleep(random.randint(40,50))

### In this project all the data came from scrapping and the crawling is used for educational purpose only.